In [ ]:
conda activate tensflow

pip install ray[tune]

pip install ipywidgets

In [ ]:
conda activate tensflow && jupyter lab --ip=0.0.0.0 --port=8888 --allow-root --no-browser --IdentityProvider.token=''

cd /home/oleg/projects/ML/Wildfire_Satelite && mlflow server --backend-store-uri mlruns --host 0.0.0.0 --port 5000

conda activate tensflow

jupyter lab --ip=0.0.0.0 --port=8888 --allow-root --no-browser --IdentityProvider.token=''

mlflow server --backend-store-uri mlruns --host 0.0.0.0 --port 5000

watch -n 1 nvidia-smi

sudo reboot now


In [3]:
'''
Проверенные параметры для YOLO-CLS:
results = model.train(
    # 🎯 ОСНОВНЫЕ
    data=IMAGE_PATH,
    epochs=10,
    imgsz=224,
    batch=16,
    device=0,
    
    # 🔧 ОПТИМИЗАТОР И LEARNING RATE
    lr0=0.01,           # Начальный learning rate
    lrf=0.01,           # Финальный learning rate (lr0 * lrf)
    momentum=0.937,      # SGD momentum
    weight_decay=0.0005, # L2 регуляризация
    warmup_epochs=3.0,   # Эпохи разогрева
    warmup_momentum=0.8, # Momentum при разогреве
    warmup_bias_lr=0.1,  # Learning rate для bias при разогреве
    
    # 📊 АУГМЕНТАЦИЯ ДАННЫХ
    augment=True,        # Применять аугментацию
    hsv_h=0.015,         # Аугментация Hue (оттенок)
    hsv_s=0.7,           # Аугментация Saturation (насыщенность)
    hsv_v=0.4,           # Аугментация Value (яркость)
    degrees=0.0,         # Вращение изображения (± градусы)
    translate=0.1,       # Смещение изображения (± доля)
    scale=0.5,           # Масштабирование изображения (± коэффициент)
    shear=0.0,           # Наклон изображения (± градусы)
    perspective=0.0,     # Перспектива изображения (± доля)
    flipud=0.0,          # Отражение по вертикали (вероятность)
    fliplr=0.5,          # Отражение по горизонтали (вероятность)
    
    # 🎛️ ДОПОЛНИТЕЛЬНЫЕ НАСТРОЙКИ
    patience=10,         # Терпение для ранней остановки
    save=True,           # Сохранять чекпоинты обучения
    save_period=-1,      # Сохранять чекпоинт каждые X эпох
    workers=8,           # Максимальное число workers для загрузки данных
    project='wildfire_classification',  # Имя проекта
    name='exp',          # Имя эксперимента
    exist_ok=False,      # Разрешить существующий проект/имя, не инкрементировать
    pretrained=True,     # Использовать предобученную модель
    optimizer='auto',    # Оптимизатор: [SGD, Adam, AdamW, RMSProP]
    verbose=True,        # Выводить подробный лог
    seed=0,              # Seed для воспроизводимости
    deterministic=True,  # Включить детерминистический режим
    single_cls=False,    # Обучать как датасет с одним классом
    image_weights=False, # Использовать взвешенный выбор изображений
    rect=False,          # Поддержка прямоугольного обучения
    cos_lr=False,        # Использовать косинусный scheduler для learning rate
    close_mosaic=10,     # Отключить мозаичную аугментацию для последних эпох
    
    # 💾 СИСТЕМНЫЕ НАСТРОЙКИ
    amp=True,            # Automatic Mixed Precision (AMP) обучение
    overlap_mask=True,   # Перекрывающиеся маски (для сегментации)
    mask_ratio=4,        # Коэффициент уменьшения маски (для сегментации)
    dropout=0.0,         # Dropout регуляризация (для классификации)
    val=True,            # Валидация/тестирование во время обучения
)
'''

"\nПроверенные параметры для YOLO-CLS:\nresults = model.train(\n    # 🎯 ОСНОВНЫЕ\n    data=IMAGE_PATH,\n    epochs=10,\n    imgsz=224,\n    batch=16,\n    device=0,\n\n    # 🔧 ОПТИМИЗАТОР И LEARNING RATE\n    lr0=0.01,           # Начальный learning rate\n    lrf=0.01,           # Финальный learning rate (lr0 * lrf)\n    momentum=0.937,      # SGD momentum\n    weight_decay=0.0005, # L2 регуляризация\n    warmup_epochs=3.0,   # Эпохи разогрева\n    warmup_momentum=0.8, # Momentum при разогреве\n    warmup_bias_lr=0.1,  # Learning rate для bias при разогреве\n\n    # 📊 АУГМЕНТАЦИЯ ДАННЫХ\n    augment=True,        # Применять аугментацию\n    hsv_h=0.015,         # Аугментация Hue (оттенок)\n    hsv_s=0.7,           # Аугментация Saturation (насыщенность)\n    hsv_v=0.4,           # Аугментация Value (яркость)\n    degrees=0.0,         # Вращение изображения (± градусы)\n    translate=0.1,       # Смещение изображения (± доля)\n    scale=0.5,           # Масштабирование изображения (± ко

In [32]:
from ultralytics import YOLO
import os
from ray import tune

IMAGE_PATH = "/home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1"

model = YOLO('yolov8n-cls.pt')
#model = YOLO('yolov8s-cls.pt')

# Явно укажите MLflow путь
os.environ['MLFLOW_TRACKING_URI'] = 'file:./mlruns'

In [36]:
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')

# Tuning конфиг
tune_config = {
    'lr0': (1e-5, 1e-2),           # диапазон для поиска
    'weight_decay': (1e-5, 1e-3),  # диапазон для поиска
}

# Запускаем TUNING
results = model.tune(
    data=IMAGE_PATH,
    epochs=5,
    iterations=10,
    space=tune_config,              # передаем конфиг
    batch=128,
    degrees=5.0,
    translate=0.1,
    fliplr=0.5,
    hsv_s=0.5,
    name='yolov8n_tuning',
    project='wildfire_classification'
)

Tuner: Initialized Tuner instance with 'tune_dir=/home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8n_tuning'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'weight_decay': 0.0005}
Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, h

Exception in thread Thread-3 (_pin_memory_loop):
Traceback (most recent call last):
  File "/home/oleg/miniconda3/envs/tensflow/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/oleg/miniconda3/envs/tensflow/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/home/oleg/miniconda3/envs/tensflow/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 52, in _pin_memory_loop
    do_one_step()
  File "/home/oleg/miniconda3/envs/tensflow/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/oleg/miniconda3/envs/tensflow/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/oleg/miniconda3/envs/tensflow/lib/python3.11/site-packages

Saved /home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8n_tuning/tune_scatter_plots.png
Saved /home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8n_tuning/tune_fitness.png

Tuner: 2/10 iterations complete ✅ (254.78s)
Tuner: Results saved to /home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8n_tuning
Tuner: Best fitness=0.99389 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/accuracy_top1': 0.98778, 'metrics/accuracy_top5': 1.0, 'val/loss': 0.03535, 'fitness': 0.99389}
Tuner: Best fitness model is /home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/train2
Printing '/home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8n_tuning/best_hyperparameters.yaml'

lr0: 0.01
weight_decay: 0.0005

Tuner: Starting iteration 3/10 with hyperparameters: {'lr0': 0.01, 'weight_decay': 0.00036}
Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine/t

In [37]:
# YOLO автоматически просканирует папку и поймет структуру
results = model.train(
    data=IMAGE_PATH,  # ✅ ВАШ ПУТЬ ЗДЕСЬ
    # Сбалансированная аугментация
    degrees=5.0,
    translate=0.1,
    fliplr=0.5,          # Отражение по горизонтали (вероятность)
    hsv_s=0.5,           # Аугментация Saturation (насыщенность)
    
    epochs=20,
    imgsz=224,
    batch=128,
    lr0=0.00837,
    weight_decay=0.00041,
    patience=10,
    save=True,
    device=0,
    plots=True,        # Создает подробные графики
    
    # Для лучшего отслеживания в MLflow
    name='yolov8s_experiment',
    project='wildfire_classification',
    
    verbose=True,
    val=True,            # Валидация/тестирование во время обучения
)

# lr0            0.00837
# weight_decay   0.00041
# 

Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00837, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_experiment18, nbs=64, nms=False, opset=None, opt

2025/10/28 11:25:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


MLflow: logging run_id(6a14c4fe33154b319f75fda76a838d8f) to file:./mlruns
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to /home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8s_experiment18
Starting training for 20 epochs...

      Epoch    GPU_mem       loss  Instances       Size
       1/20      1.56G     0.2176         41        224: 100% ━━━━━━━━━━━━ 237/237 18.7it/s 12.7s<0.1s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 25/25 20.5it/s 1.2s0.1s
                   all      0.973          1

      Epoch    GPU_mem       loss  Instances       Size
       2/20       1.8G    0.09681         41        224: 100% ━━━━━━━━━━━━ 237/237 23.0it/s 10.3s0.1s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 25/25 41.5it/s 0.6s0.0s
                   all      0.981          1

      Epoch    GPU_mem       loss  Instances       Size
       3/20       1.8G    0.08822   

In [ ]:
cd /home/oleg/projects/ML/Wildfire_Satelite && mlflow server --backend-store-uri runs/mlflow --host 0.0.0.0 --port 5000

In [20]:
# Загружаем обученную модель
model = YOLO('/home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8s_experiment16/weights/best.pt')

# Смотрим классы
print("🎯 Классы модели:")
print(f"Количество классов: {model.names}")
print(f"Словарь классов: {model.names}")
print(f"ID -> Имя: {dict(enumerate(model.names))}")

# Альтернативные способы
print(f"\n📋 Все атрибуты модели:")
print(f"model.names: {model.names}")
print(f"model.model.names: {getattr(model.model, 'names', 'Not found')}")

🎯 Классы модели:
Количество классов: {0: 'nowildfire', 1: 'wildfire'}
Словарь классов: {0: 'nowildfire', 1: 'wildfire'}
ID -> Имя: {0: 0, 1: 1}

📋 Все атрибуты модели:
model.names: {0: 'nowildfire', 1: 'wildfire'}
model.model.names: {0: 'nowildfire', 1: 'wildfire'}


In [21]:
from IPython.display import Image, display
import os

results_dir = "/home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8s_experiment16/weights/best.pt"

print("📈 Готовые графики из папки обучения:")

# Показываем results.png
results_path = os.path.join(results_dir, 'results.png')
if os.path.exists(results_path):
    print("✅ results.png: Общие метрики обучения")
    display(Image(filename=results_path, width=1000))

# Показываем confusion matrix
cm_path = os.path.join(results_dir, 'confusion_matrix.png')
if os.path.exists(cm_path):
    print("✅ confusion_matrix.png: Матрица ошибок")
    display(Image(filename=cm_path, width=800))

# Показываем примеры изображений
train_batch_path = os.path.join(results_dir, 'train_batch0.jpg')
if os.path.exists(train_batch_path):
    print("✅ train_batch0.jpg: Примеры обучающих изображений")
    display(Image(filename=train_batch_path, width=800))

📈 Готовые графики из папки обучения:


In [38]:
from ultralytics import YOLO

# Загружаем модель
model = YOLO('/home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8s_experiment18/weights/best.pt')

# Тестируем на тестовых данных
results = model.val(data=IMAGE_PATH, split='test', verbose=True)

print("\n" + "="*50)
print("📊 ОСНОВНЫЕ МЕТРИКИ")
print("="*50)
print(f"🎯 Top-1 Accuracy: {results.top1:.4f}")
print(f"📈 Top-5 Accuracy: {results.top5:.4f}")
print("="*50)

Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/train... found 30250 images in 2 classes ✅ 
val: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/valid... found 6300 images in 2 classes ✅ 
test: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test... found 6300 images in 2 classes ✅ 
test: Fast image access ✅ (ping: 0.3±0.3 ms, read: 89.5±39.6 MB/s, size: 28.2 KB)
test: Scanning /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test... 6299 images, 1 corrupt: 100% ━━━━━━━━━━━━ 6299/6299 9.4Mit/s 0.0s
test: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test/wildfire/-73.15884,46.38819.jpg:

In [39]:
from ultralytics import YOLO

# Загружаем модель
model = YOLO('/home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8s_experiment18/weights/best.pt')

# Тестируем на тестовых данных
results = model.val(data=IMAGE_PATH, split='test', verbose=False)

print("\n" + "="*50)
print("📊 ВСЕ МЕТРИКИ ИЗ RESULTS")
print("="*50)

# Основные метрики точности
print(f"🎯 Top-1 Accuracy: {results.top1:.4f}")
print(f"📈 Top-5 Accuracy: {results.top5:.4f}")

# Если есть другие атрибуты
print(f"📊 Speed: {results.speed}")

# Выводим все доступные атрибуты
print(f"\n🔍 Все доступные атрибуты:")
for attr in dir(results):
    if not attr.startswith('_'):  # Только публичные атрибуты
        try:
            value = getattr(results, attr)
            if not callable(value):  # Только не-функции
                print(f"  {attr}: {value}")
        except:
            pass

print("="*50)

Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/train... found 30250 images in 2 classes ✅ 
val: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/valid... found 6300 images in 2 classes ✅ 
test: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test... found 6300 images in 2 classes ✅ 
test: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3377.9±1710.6 MB/s, size: 28.2 KB)
test: Scanning /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test... 6299 images, 1 corrupt: 100% ━━━━━━━━━━━━ 6299/6299 15.2Mit/s 0.0s
test: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test/wildfire/-73.15884,46.38819

In [40]:
from ultralytics import YOLO
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
from tqdm import tqdm
import os

# Загружаем модель
model = YOLO('/home/oleg/projects/ML/Wildfire_Satelite/wildfire_classification/yolov8s_experiment18/weights/best.pt')

# Тестируем для получения accuracy
results = model.val(data=IMAGE_PATH, split='test', verbose=False)

# Собираем данные для F1, Precision, Recall
all_predictions = []
all_true_labels = []

test_path = os.path.join(IMAGE_PATH, 'test')

print("🔄 Сбор данных для F1-score...")

# Считаем общее количество изображений
total_images = 0
for class_name in ['nowildfire', 'wildfire']:
    class_path = os.path.join(test_path, class_name)
    if os.path.exists(class_path):
        total_images += len([f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])

# Обрабатываем с прогресс баром
with tqdm(total=total_images, desc="Сбор предсказаний") as pbar:
    for class_name in ['nowildfire', 'wildfire']:
        class_path = os.path.join(test_path, class_name)
        if os.path.exists(class_path):
            for img_file in os.listdir(class_path):
                if img_file.lower().endswith(('.jpg', '.png', '.jpeg')):
                    img_path = os.path.join(class_path, img_file)
                    
                    try:
                        results_single = model(img_path, verbose=False)
                        pred_class = results_single[0].probs.top1
                        true_class = 0 if class_name == 'nowildfire' else 1
                        
                        all_predictions.append(pred_class)
                        all_true_labels.append(true_class)
                    
                    except Exception as e:
                        continue
                    
                    pbar.update(1)

# Вычисляем метрики
f1 = f1_score(all_true_labels, all_predictions)
precision = precision_score(all_true_labels, all_predictions)
recall = recall_score(all_true_labels, all_predictions)

print("\n" + "="*50)
print("📊 ПОЛНЫЕ МЕТРИКИ КЛАССИФИКАЦИИ")
print("="*50)
print(f"🎯 Top-1 Accuracy:  {results.top1:.4f}")
print(f"📈 Top-5 Accuracy:  {results.top5:.4f}")
print(f"⚖️  F1-Score:        {f1:.4f}")
print(f"📊 Precision:       {precision:.4f}")
print(f"📉 Recall:          {recall:.4f}")
print("="*50)

# Анализ ошибок
false_positives = np.sum((np.array(all_predictions) == 1) & (np.array(all_true_labels) == 0))
false_negatives = np.sum((np.array(all_predictions) == 0) & (np.array(all_true_labels) == 1))

print(f"\n❌ Анализ ошибок:")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")

if false_negatives == 0:
    print("✅ Отлично! Пропущенных пожаров: 0")

Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/train... found 30250 images in 2 classes ✅ 
val: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/valid... found 6300 images in 2 classes ✅ 
test: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test... found 6300 images in 2 classes ✅ 
test: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3062.0±1412.2 MB/s, size: 28.2 KB)
test: Scanning /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test... 6299 images, 1 corrupt: 100% ━━━━━━━━━━━━ 6299/6299 17.8Mit/s 0.0s
test: /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test/wildfire/-73.15884,46.38819

Сбор предсказаний:  93%|███████████████████████████████████████████████████████    | 5874/6300 [00:22<00:01, 332.51it/s]

WARNING ⚠️ Image Read Error /home/oleg/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test/wildfire/-73.15884,46.38819.jpg


Сбор предсказаний: 100%|██████████████████████████████████████████████████████████▉| 6299/6300 [00:23<00:00, 263.95it/s]



📊 ПОЛНЫЕ МЕТРИКИ КЛАССИФИКАЦИИ
🎯 Top-1 Accuracy:  0.9946
📈 Top-5 Accuracy:  1.0000
⚖️  F1-Score:        0.9951
📊 Precision:       0.9974
📉 Recall:          0.9928

❌ Анализ ошибок:
False Positives: 9
False Negatives: 25
